### Bar Chart Race

In [ ]:
import os ; print(os.getcwd())
import pandas as pd 
import numpy as np  

import matplotlib.pyplot as plt  
import matplotlib.ticker as ticker
import matplotlib.font_manager 

from matplotlib import rc, font_manager
from mpl_toolkits.axes_grid1 import Divider, Size  

from matplotlib.offsetbox import (OffsetImage, AnnotationBbox)

In [ ]:
from string import ascii_lowercase as letter

all_letters = []
n = len(letter)
for i in range(0, n):
    for j in range(0, n):
        for k in range(0, n):
            for l in range(0, n):
                all_letters.append(letter[i] +letter[j] + letter[k] + letter[l] )

print( all_letters[:5] )
print(len(all_letters) )


In [ ]:
df = pd.read_excel('C:/Users/riheiman/Documents/Python/bar_chart_animation/Cumulative Revenue by Material (lvl 2) - Monthly(2022).xlsx', sheet_name = 'sheet1')
df['YEAR'] = df['YEAR'].str.replace('FY', '')
df['YEAR'] = df['YEAR'].str[:3] + ', ' + df['YEAR'].str[-4:]
df = df.set_index('YEAR') ; n_row, n_col = df.shape
df = df.replace(" ", np.nan )
df = df.ffill(axis = 0)
df = df.fillna(0.0) 
df = df.apply(pd.to_numeric)
print(df.dtypes) 
print('ROW TOTAL = ' + str(n_row) + ', COL TOTAL = ' + str(n_col))
df.head(114)      

----
## Plotting

In [ ]:
def prepare_data(df, steps=10):  
    df = df.reset_index()
    df.index = df.index * steps
    last_idx = df.index[-1] + 1
    df_expanded = df.reindex(range(last_idx))
    df_expanded['YEAR'] = df_expanded['YEAR'].fillna(method='ffill')
    df_expanded = df_expanded.set_index('YEAR')
    df_rank_expanded = df_expanded.rank(axis=1, method='first')
    df_expanded = df_expanded.interpolate()
    df_rank_expanded = df_rank_expanded.interpolate()
    return df_expanded, df_rank_expanded

df_expanded, df_rank_expanded = prepare_data(df, steps = 60 )         

In [ ]:
rank =  df_rank_expanded.iloc[0].sort_values(ascending=False)[:5] 
values =  df_expanded.iloc[0].sort_values(ascending=False)[:5].divide(1000000) 
product_name = df_rank_expanded.iloc[0].sort_values(ascending=False)[:20].index
#print(rank) 
#print(values.values.round(decimals=2))
print(product_name)  

In [ ]:
corporate_colors = {
    'Slice':'#C69214',  # mustard
    'Shred':'#840029',  # red
    'Ref Snack': '#48D1E0' , # light blue
    'Frozen': '#BEC6D0',  # light gray
    'Non-Ref Snk': '#74aa50' , # green
    'Sauce': '#723ef7' , # purple 
    'Chunk': '#47515F' , # dark gray
    'Bal Breaks': '#C26232' # dark orange 
    }

# get photos for products
photos_folder = os.listdir('photos') ; all_photos = []
for i in photos_folder:
    if '.png' in (i):
        all_photos.append(i)
print(all_photos)

top_n = 8
center_size = 20 
v_line_color = "#acacad"

font_name = 'Calibri'
font = {'fontname':font_name}  # Calibri  
ticks_font = font_manager.FontProperties(family = font_name, style='normal', size=12, weight='normal', stretch='normal') 

In [ ]:
# [f.name for f in matplotlib.font_manager.fontManager.ttflist] 
n_row, n_col = df_expanded.shape
print(n_row) ; print(113*10)      

In [ ]:
n_row, n_col = df_expanded.shape
keep_top = 8

#for i in range(0,1):
for i in range(0, n_row):

    current_year = str(df_expanded.index[i])

    current_letter = all_letters[i]
    
    rank =  df_rank_expanded.iloc[i].sort_values(ascending=False)[:top_n] 
    lowest_rank = np.round( np.min(rank), decimals=0)

    values =  df_expanded.iloc[i].sort_values(ascending=False)[:top_n].divide(1000000) 

    product_name = df_rank_expanded.iloc[i].sort_values(ascending=False)[:top_n].index
    
    products = product_name.map(lambda x: str(x).ljust(center_size)) ;  l = len(products)

    print('The Current Year is = ' + str(current_year) + ', plot name = ' + str(current_letter) + ', i = ' + str(i) + ' Lowest Rank  = ' + str(lowest_rank) + ', highest rank = ' + str(np.max(rank)))
    
    with plt.style.context('seaborn-white'):  

        #fig, ax = plt.subplots( figsize = (10.5 , 7.0) )
        fig, ax = plt.subplots( figsize = (9 , 5 ) )
        plt.gcf().subplots_adjust(left = .10 )

        # Horizontal Bar Plot. Fix colors to categories
        ax.barh(y = rank , width = values , tick_label = product_name , color = [corporate_colors[i] for i in product_name] )

        ax.grid(True, axis = 'x', linewidth = 1.2, color = v_line_color, alpha = .40)   
        

        plt.xlabel('Revenue (Millions)', fontsize = 14, **font)
        plt.title('Cumulative Cheese Sales (2022) \n', fontsize = 16, **font) 

        
        # Increase font size for x and y labels  
        for label in ax.get_xticklabels():
            label.set_fontproperties(ticks_font)

        for label in ax.get_yticklabels():
            label.set_fontproperties(ticks_font)
        
        # change x-axis labels to be thousands of dollars
        formatter = ticker.FuncFormatter( lambda x, pos: "${:,.0f}".format(x) )
        plt.gca().xaxis.set_major_formatter(formatter)


        y_lim_min, y_lim_max = 0.5, 8.5  
        plt.ylim(y_lim_min , y_lim_max )  
        x_low = ax.get_xlim()[0] ; x_high = ax.get_xlim()[1] ; new_x_max = x_high+210
        ax.set_xlim( x_low, new_x_max ) 

        #  Display the current year    
        x = (new_x_max*.75) ; ax.text( x, 1.5, current_year, fontsize = 24, **font  )  

        #  Insert Images
        for i, product in enumerate(product_name):
            full_path = 'photos/' + str(product) + '.png'
            try:
                img = plt.imread(full_path)
                imagebox = OffsetImage(img, zoom = 0.08 ) 
                
                ab = AnnotationBbox(imagebox, (values[i] + (new_x_max * .08) , rank[i]) , frameon=False )
                ax.add_artist(ab) 

            except FileNotFoundError: 
                pass
                #print('File Not Found: ' + str(full_path))   
            

        #  Label the bar with numbers            
        for i in range( 0, top_n ):   
            text =  int( np.ceil( values[i]) )    
            text = "${:,.0f}".format( text )  
            ax.text(values[i] + (new_x_max * .15) , rank[i]-.25,  text , fontsize = 10)           
        
        # remove the frame  
        ax.spines['top'].set_visible(False)    
        ax.spines['right'].set_visible(False)  
        ax.spines['bottom'].set_visible(False) 
        ax.spines['left'].set_visible(False)    

        plt.savefig('plots/movie_monthly/' + str(current_letter) + '.jpg', dpi = 450 )       
        plt.close()
        #plt.show()         



---
### Appendix: All Products

